In [1]:

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)







In [2]:
def get_DIE_at_offset(CU, offset):
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None

def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_pointer_type':
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU,_attr.value) 
        
    
    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")
    
    


In [3]:
from collections import defaultdict
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()

    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()

In [4]:
def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)

In [8]:
# https://github.com/eliben/pyelftools/blob/master/examples/dwarf_location_info.py


from elftools.dwarf.locationlists import LocationParser, LocationExpr
import os
def parse_dwarf_to_get_func_params(filename):
    FUNC_PARAMS_DICT = {}
    FUNC_PARAMS_DICT['uesrdef_datastructs'] = {}
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()
        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()
        

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        
        
        for CU in dwarfinfo.iter_CUs():
            CU_DIR_PATH = None
            CU_FILENAME = None
            for attr in CU.get_top_DIE().attributes.values():
                if attr.name == 'DW_AT_comp_dir':
                    CU_DIR_PATH = (attr.value.decode("utf-8"))
                if attr.name == 'DW_AT_name'    :
                    CU_FILENAME = (attr.value.decode("utf-8"))

            line_program = dwarfinfo.line_program_for_CU(CU)

         
            CU_dictionary_key = os.path.join(CU_DIR_PATH, CU_FILENAME)
            if CU_dictionary_key not in FUNC_PARAMS_DICT:
                FUNC_PARAMS_DICT[CU_dictionary_key] = {}
            


            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = None
            for DIE in CU.iter_DIEs():
                
                ############################################################
                #############   Prasing Function DIEs start ################

                
                if DIE.tag == 'DW_TAG_subprogram':
                    if 'DW_AT_low_pc' in DIE.attributes and 'DW_AT_high_pc' in DIE.attributes :
                        low_pc = DIE.attributes['DW_AT_low_pc'].value
                        high_pc = DIE.attributes['DW_AT_high_pc'].value
                        #todo use low hi as boundary and use instead of connected comps
                        print("Low PC: ",hex(low_pc) , " High PC" , hex(high_pc))
                    else:
                        pass
#                         print("NO PC given")
                    are_DIEs_of_function = True
                    
                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            if FUNC_name not in FUNC_PARAMS_DICT[CU_dictionary_key]:
                                FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name] ={}
#                             print("SUBPROGRAM: ",FUNC_name)
                            
                if DIE.tag in[ 'DW_TAG_formal_parameter','DW_TAG_variable' , 'DW_TAG_member']:
            
                    if DIE.tag == 'DW_TAG_member':
                        print("DBG#DW_TAG_member")
                        print(DIE)
                    tags = [attr.name for attr in DIE.attributes.values()]
                    PARAM_name = None
                    if FUNC_name==None:
                        
                        FUNC_name ="global"
                        
                        if FUNC_name not in FUNC_PARAMS_DICT[CU_dictionary_key]:
                            FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name]={}
                        
                    if "DW_AT_name" in tags:
                        
                        die_dict = {}
                        
                        for attr in DIE.attributes.values():
                            die_dict[attr.name] = attr
                        
                        PARAM_name = die_dict['DW_AT_name'].value.decode("utf-8")
                        
                        if PARAM_name not in FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name]:
                            FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name][PARAM_name] = {}
                        var_type = DIE.tag.split('_')[-1]
                        FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,die_dict['DW_AT_type'].value) , 'kind':var_type}
                        
                        if 'DW_AT_location' in die_dict:

                            try:
                                loc = loc_parser.parse_from_attribute(die_dict['DW_AT_location'],
                                                                      CU['version'])
                                
#                                 print(CU_dictionary_key,FUNC_name,PARAM_name)
                                if isinstance(loc, LocationExpr):
                                    loc_info_str = describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset)
                                    offset_temp = (loc_info_str.split('-')[-1]).split(')')[0]
#                                     print('1a ',loc_info_str, offset_temp)
#                                     print('1b ', PARAM_name,loc_info_str, int(offset_temp)-LOCATION_SUBSTRACT_FACTOR)
                                    FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= loc_info_str

                                elif isinstance(loc, list):
#                                     print(PARAM_name,show_loclist(loc,dwarfinfo,'      ', CU.cu_offset))
                                    FUNC_PARAMS_DICT[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= show_loclist(loc,
                                                       dwarfinfo,'      ', CU.cu_offset)
                            except:

                                print("ERROR",DIE)
                                pass
#                 if DIE.tag=='DW_TAG_member':
                    
#                         print("DBG#DW_TAG_member")
#                         print(DIE)
#                         print(FUNC_PARAMS_DICT)
                    
                if DIE.tag =='DW_TAG_global_variable':
                    print("DBG# DW_TAG_global_variable")
                    print(DIE)
                    pass #TODO
                if DIE.tag in ['DW_TAG_structure_type' , 'DW_TAG_union_type' , 'DW_TAG_enumeration_type']:
                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name":
                            FUNC_PARAMS_DICT['uesrdef_datastructs'][attr.value.decode("utf-8")]=DIE.tag
                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
    
    
    return FUNC_PARAMS_DICT



filename = './../../binaries/c_many/stacktest'
FUNC_PARAMS = parse_dwarf_to_get_func_params(filename)

Processing file: ./../../binaries/c_many/stacktest
DBG#DW_TAG_member
DIE DW_TAG_member, size=15, has_children=False
    |DW_AT_name        :  AttributeValue(name='DW_AT_name', form='DW_FORM_strp', value=b'number', raw_value=13750, offset=165)
    |DW_AT_decl_file   :  AttributeValue(name='DW_AT_decl_file', form='DW_FORM_data1', value=2, raw_value=2, offset=169)
    |DW_AT_decl_line   :  AttributeValue(name='DW_AT_decl_line', form='DW_FORM_data1', value=2, raw_value=2, offset=170)
    |DW_AT_decl_column :  AttributeValue(name='DW_AT_decl_column', form='DW_FORM_data1', value=9, raw_value=9, offset=171)
    |DW_AT_type        :  AttributeValue(name='DW_AT_type', form='DW_FORM_ref4', value=105, raw_value=105, offset=172)
    |DW_AT_data_member_location:  AttributeValue(name='DW_AT_data_member_location', form='DW_FORM_block1', value=[35, 0], raw_value=[35, 0], offset=176)

DBG#DW_TAG_member
DIE DW_TAG_member, size=15, has_children=False
    |DW_AT_name        :  AttributeValue(name='DW_AT_n

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
FUNC_PARAMS 


In [9]:
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment



filePath = './../../binaries/c_many/stacktest'


In [10]:
# https://www.capstone-engine.org/lang_python.html



from capstone import *

from capstone.x86 import *


address_inst = {}
with open(filePath, 'rb') as f:
    elf = ELFFile(f)
    code = elf.get_section_by_name('.text')
    ops = code.data()
    print('code data size: ',code.data_size)
    addr = code['sh_addr']
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    for inst in md.disasm(ops, addr):
        
        address_inst[hex(inst.address)] = inst
        

        print('\n'*3)
        print(inst.address, inst.mnemonic+"  "+inst.op_str)
        (regs_read, regs_write) = inst.regs_access()

        if len(inst.operands) > 0:
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                    print(hex(o.value.mem.disp),o.value.mem.disp)



code data size:  2073




4320 endbr64  




4324 xor  ebp, ebp




4326 mov  r9, rdx




4329 pop  rsi




4330 mov  rdx, rsp




4333 and  rsp, 0xfffffffffffffff0




4337 push  rax




4338 push  rsp




4339 xor  r8d, r8d




4342 xor  ecx, ecx




4344 lea  rdi, [rip + 0x517]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x517
0x517 1303




4351 call  qword ptr [rip + 0x2ed3]
		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].mem.disp: 0x2ed3
0x2ed3 11987




4357 hlt  




4358 nop  word ptr cs:[rax + rax]
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.index: REG = rax
0x0 0




4368 lea  rdi, [rip + 0x2f01]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2f01
0x2f01 12033




4375 lea  rax, [rip + 0x2efa]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2efa
0x2efa 12026




4382 cmp  rax, rdi




4385 je  0x1138


In [11]:
from collections import defaultdict
import posixpath


In [12]:
addr_lineProgram ={}
addr_sourceFile = {}
with open(filePath, 'rb') as f:
    elffile = ELFFile(f)

    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        exit(0)

    dwarfinfo = elffile.get_dwarf_info()
    for CU in dwarfinfo.iter_CUs():
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue

        # Print a reverse mapping of filename -> #entries
        filename_map = line_entry_mapping(line_program)
        for line_entry in line_program.get_entries():

            if line_entry.state!=None:
                addr_lineProgram[hex(line_entry.state.address)] = line_entry
                addr_sourceFile [hex(line_entry.state.address)] = filename_map

        
        


  Found a compile unit at offset 0, length 986
  Found a compile unit at offset 990, length 1196
  Found a compile unit at offset 2190, length 484


In [13]:
# print(addr_lineProgram)
# address_inst

def getSource(sourceFileName, row , col):
    basePath = "/home/nahid/reverse/binaries/c_many/"
    sourceFilePath = os.path.join(basePath , sourceFileName)
    sourceFile = open(sourceFilePath, "r")
    fileContent = sourceFile.readlines()
    
    row_content =  fileContent[row-1]
    row_content = row_content[:(col-1)] + '|'+row_content[(col-1)]+'|' +row_content[col:]
    
    return row_content


In [14]:

# FUNCTION_DECL
# https://stackoverflow.com/questions/43460605/function-boundary-identification-using-libclang
# https://eli.thegreenplace.net/2011/07/03/parsing-c-in-python-with-clang


import clang.cindex



def get_all_var_types(source_path):
    srcFileName = source_path.split('/')[-1]

    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.VAR_DECL:
            # print(dir(f))
            print(f.extent.start.file.name)
            originFileName = f.extent.start.file.name.split('/')[-1]
            print(originFileName)
            print(f.displayname)
            print(f.type.spelling)
            print(f.extent.start.line,f.extent.start.column)
            print('\n')
            
            

   

def get_all_function_types(source_path):
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:
            # print(dir(f))
            print(f.displayname)
            print('function name: ',( f.spelling))
            print('Returns: ',(f.result_type.spelling))
            
            
            arg_len = len(list(f.type.argument_types()))
            if arg_len>0:
                arg_types = list(f.type.argument_types())
                for arg_type in arg_types:
                    print('arg_type:',arg_type.spelling)
                args = list(f.get_arguments())
                for arg in args:
                    print('arg:',arg.spelling)


            print("\n\n\n")

            
def get_function_boundaries(source_path):
    
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:

            function_name = f.displayname.split('(')[0]
            function_boundary_by_name[function_name]={}
            function_boundary_by_name[function_name] = { 'src_path':f.extent.start.file.name,
                              'src_file':f.extent.start.file.name.split('/')[-1],
                              'start_line':f.extent.start.line,
                              'start_col':f.extent.start.column,
                              'end_line':f.extent.end.line,
                              'end_col':f.extent.end.column}
    return function_boundary_by_name

def get_containing_function(source_file_path, line, col=0):
    function_boundary_by_name = get_function_boundaries(source_file_path)
    
    for function_name, item in function_boundary_by_name.items():
        if item['src_path'] == source_file_path:
            if line>= item['start_line'] and line<= item['end_line']:
                return function_name
        

# _=get_all_var_types('/home/nahid/reverse/binaries/c_many/stack.c' )
# print(_)

_=get_all_function_types('/home/nahid/reverse/binaries/c_many/stack.c' )
print(_)

remove(const char *)
function name:  remove
Returns:  int
arg_type: const char *
arg: __filename




rename(const char *, const char *)
function name:  rename
Returns:  int
arg_type: const char *
arg_type: const char *
arg: __old
arg: __new




renameat(int, const char *, int, const char *)
function name:  renameat
Returns:  int
arg_type: int
arg_type: const char *
arg_type: int
arg_type: const char *
arg: __oldfd
arg: __old
arg: __newfd
arg: __new




fclose(FILE *)
function name:  fclose
Returns:  int
arg_type: FILE *
arg: __stream




tmpfile()
function name:  tmpfile
Returns:  FILE *




tmpnam(char *)
function name:  tmpnam
Returns:  char *
arg_type: char[20]
arg: 




tmpnam_r(char *)
function name:  tmpnam_r
Returns:  char *
arg_type: char[20]
arg: __s




tempnam(const char *, const char *)
function name:  tempnam
Returns:  char *
arg_type: const char *
arg_type: const char *
arg: __dir
arg: __pfx




fflush(FILE *)
function name:  fflush
Returns:  int
arg_type: FILE *
arg: __s

In [17]:

dir_path = './../../binaries/c_many/'

with open('stacktest.s', 'w') as outFile:
    # outFile.write('file contents\n')
    lastSource = ""
    for address in address_inst:
        inst = address_inst[address]
        instrctionCode = (address+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)
        
        
        OFFSET = None
        if len(inst.operands) > 0:
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                        OFFSET = o.value.mem.disp
                    print(hex(o.value.mem.disp),o.value.mem.disp)
                    
        
        if address in addr_lineProgram:
            line = addr_lineProgram[address]

            srcFileName =list(addr_sourceFile[address].keys())[0] #TODO not single file always 
            
            if srcFileName!=lastSource:
                outFile.write("\n"+ '#'*100+"\n"+ srcFileName.rjust(45) +'\n'+'#'*100+ "\n\n")
                lastSource = srcFileName
            
            sourceCode = getSource(srcFileName,addr_lineProgram[address].state.line, addr_lineProgram[address].state.column)
            function_name = get_containing_function(dir_path+srcFileName ,addr_lineProgram[address].state.line , addr_lineProgram[address].state.column)
            print(function_name)
            
            if '\n' not in  sourceCode:
                sourceCode+=sourceCode+"\n"
            outFile.write(instrctionCode+"#"+ sourceCode  )
            print(instrctionCode+"#"+ sourceCode)
            
        else:
            
            outFile.write(instrctionCode+ '\n'  )
            print(instrctionCode)
        if OFFSET:
            outFile.write("MEMORY OFFSET:     "+str(hex(OFFSET))+"     "+str(OFFSET)+'\n')
            pass

0x10e0:	endbr64                              
0x10e4:	xor ebp, ebp                         
0x10e6:	mov r9, rdx                          
0x10e9:	pop rsi                              
0x10ea:	mov rdx, rsp                         
0x10ed:	and rsp, 0xfffffffffffffff0          
0x10f1:	push rax                             
0x10f2:	push rsp                             
0x10f3:	xor r8d, r8d                         
0x10f6:	xor ecx, ecx                         
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x517
0x517 1303
0x10f8:	lea rdi, [rip + 0x517]               
		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].mem.disp: 0x2ed3
0x2ed3 11987
0x10ff:	call qword ptr [rip + 0x2ed3]        
0x1105:	hlt                                  
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.index: REG = rax
0x0 0
0x1106:	nop word ptr cs:[rax + rax]          
		operands[1].type: MEM
			operands[1].mem.base: R

dummy
0x12a7:	mov rax, qword ptr [rbp - 0x98]      #        |p|rintf('%s',char_pointer);

0x12ae:	mov rsi, rax                         
0x12b1:	mov edi, 0x2573                      
0x12b6:	mov eax, 0                           
0x12bb:	call 0x10b0                          
		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-6c
-0x6c -108
dummy
0x12c0:	mov dword ptr [rbp - 0x6c], 0        #        int x1,x2,x3,x4,x5,|x|6 = 0;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-78
-0x78 -120
dummy
0x12c7:	mov dword ptr [rbp - 0x78], 0        #        for (int |k|=0; k<20;k++){

dummy
0x12ce:	jmp 0x12e4                           #        |f|or (int k=0; k<20;k++){

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-74
-0x74 -116
dummy
0x12d0:	mov eax, dword ptr [rbp - 0x74]      #            int_arr[k] = e_int|%|k;

0x12d3:	cdq                                  
		operands[0].type: ME

test_structs
0x13db:	lea rax, [rbp - 0x60]                #    |s|trcpy(s1.name, "Kamlesh Joshi");

0x13df:	add rax, 4                           
0x13e3:	movabs rdx, 0x206873656c6d614b       
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
0x0 0
0x13ed:	mov qword ptr [rax], rdx             
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.disp: 0x8
0x8 8
0x13f0:	mov dword ptr [rax + 8], 0x68736f4a  
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.disp: 0xc
0xc 12
0x13f7:	mov word ptr [rax + 0xc], 0x69       
		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-60
-0x60 -96
test_structs
0x13fd:	lea rax, [rbp - 0x60]                #    |s|trcpy(s1.branch, "Computer Science And Engineering");

0x1401:	add rax, 0x22                        
0x1405:	movabs rdx, 0x72657475706d6f43       
0x140f:	movabs rcx, 0x65636e6569635320       
		operands[0].type: MEM
			operands[0].mem.base: REG = ra

printValue
0x15c0:	mov esi, eax                         #        |p|rintf("%d\n", o->value.i);

		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xa84
0xa84 2692
0x15c2:	lea rax, [rip + 0xa84]               
0x15c9:	mov rdi, rax                         
0x15cc:	mov eax, 0                           
0x15d1:	call 0x10b0                          
printValue
0x15d6:	jmp 0x1613                           #        |b|reak;

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-8
-0x8 -8
printValue
0x15d8:	mov rax, qword ptr [rbp - 8]         #        printf("%.3f", o->value|.|f);

		operands[1].type: MEM
			operands[1].mem.base: REG = rax
			operands[1].mem.disp: 0x4
0x4 4
0x15dc:	movss xmm0, dword ptr [rax + 4]      
printValue
0x15e1:	pxor xmm1, xmm1                      #        |p|rintf("%.3f", o->value.f);

0x15e5:	cvtss2sd xmm1, xmm0                  
0x15e9:	movq rax, xmm1                       
0x15ee:	movq xmm0, rax 

main
0x1752:	lea rax, [rbp - 0x38]                #    |p|rintf("%d\n",pop(&stk));

0x1756:	mov rdi, rax                         
0x1759:	call 0x130a                          
0x175e:	mov esi, eax                         
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x8e6
0x8e6 2278
0x1760:	lea rax, [rip + 0x8e6]               
0x1767:	mov rdi, rax                         
0x176a:	mov eax, 0                           
0x176f:	call 0x10b0                          
		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-38
-0x38 -56
main
0x1774:	lea rax, [rbp - 0x38]                #    |p|rintf("%d\n",pop(&stk));

0x1778:	mov rdi, rax                         
0x177b:	call 0x130a                          
0x1780:	mov esi, eax                         
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x8c4
0x8c4 2244
0x1782:	lea rax, [rip + 0x8c4]               
0x1789:	mov rdi, rax    

substract
0x18d3:	mov eax, dword ptr [rbp - 4]         #    return a|-|b;

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-8
-0x8 -8
0x18d6:	sub eax, dword ptr [rbp - 8]         
substract
0x18d9:	pop rbp                              #|}|

0x18da:	ret                                  
substractf
0x18db:	endbr64                              #float substractf(float a, float b)|{|

0x18df:	push rbp                             
0x18e0:	mov rbp, rsp                         
		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-4
-0x4 -4
0x18e3:	movss dword ptr [rbp - 4], xmm0      
		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-8
-0x8 -8
0x18e8:	movss dword ptr [rbp - 8], xmm1      
		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-4
-0x4 -4
substractf
0x18ed:	movss xmm0, dword ptr [rbp - 4]      #    return a|-|b;

		operands[1].type: MEM
			o

In [ ]:
        CU_DIR_PATH = None
        CU_FILENAME = None
        for attr in CU.get_top_DIE().attributes.values():
            if attr.name == 'DW_AT_comp_dir':
                CU_DIR_PATH = attr.value.decode("utf-8")
            if attr.name == 'DW_AT_name':
                CU_FILENAME = attr.value.decode("utf-8")